# PLDM TwoRooms (L1 vs L2)

Upload the tarball created by `PLDM_hieral/colab_pack.sh`, or clone the repo manually.

In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

OUTPUT_ROOT = './WM-JEPA' 
os.makedirs(OUTPUT_ROOT, exist_ok=True)

# --- 1. ウィジェットと出力エリアの作成 ---
uploader = widgets.FileUpload(
    accept='.tar.gz,.tar',
    multiple=False
)
# ログを表示するための専用エリア
log_output = widgets.Output()

# --- 2. アップロード後に実行したい処理を関数にする ---
def process_uploaded_file(change):
    # change['new'] に新しい値が入っています
    if not change['new']:
        return

    with log_output:
        clear_output() # 前のログがあれば消す
        print("📥 ファイルを検知しました。処理を開始します...")
        
        # アップロードファイルの取得 (v8系対応)
        uploaded_file = change['new'][0]
        filename = uploaded_file['name']
        content = uploaded_file['content']
        
        # 保存
        with open(filename, "wb") as f:
            f.write(content)
        print(f"✅ 保存完了: {filename}")

        # 解凍処理
        if filename.endswith(('.tar.gz', '.tar', '.tgz')):
            print(f"📦 解凍中: {filename} ...")
            os.system(f"tar -xzf {filename} 2>&1 | grep -v 'Ignoring unknown extended header'")
            print("🎉 すべての処理が完了しました！")
        else:
            print("⚠️ tarファイルではありません。")

# --- 3. ウィジェットに「変化があったら関数を実行せよ」と命令する ---
uploader.observe(process_uploaded_file, names='value')

# --- 4. 画面に表示 ---
print("↓ ここにファイルをアップロードしてください（処理は自動で始まります）")
display(uploader, log_output)

↓ ここにファイルをアップロードしてください（処理は自動で始まります）


FileUpload(value=(), accept='.tar.gz,.tar', description='Upload')

Output()

In [19]:
!sed -i 's/self.model = torch.compile(self.model)/# self.model = torch.compile(self.model)/' */train.py

In [3]:
!pip install -q torch==2.5.1 torchvision==0.20.1 gymnasium==1.0.0 gym==0.23.1 numpy==1.26.4 d4rl==1.1 mujoco==3.2.6 mujoco-py==2.1.2.14 wandb tqdm zarr arm-pytorch-utilities
!pip install -q gdown omegaconf statsmodels

In [4]:
import os
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

CUDA available: True
GPU: NVIDIA L4


In [5]:
!bash -c "cd pldm_envs/wall && bash presaved_datasets/download_all.sh"
!bash -c "cd pldm_envs/wall && mkdir -p presaved_datasets/rendered"
!bash -c "cd pldm_envs/wall && PYTHONPATH=../.. python render_images.py --input_path presaved_datasets/ds_size_634_no_images.npz --output_path presaved_datasets/rendered/ds_size_634.npz --config configs/good_quality_data.yaml"

Downloading...
From (original): https://drive.google.com/uc?id=1NwR-ui-akIgR2xcoJHYiHjOk9U5bYCa0
From (redirected): https://drive.google.com/uc?id=1NwR-ui-akIgR2xcoJHYiHjOk9U5bYCa0&confirm=t&uuid=b93b3642-d67d-4f28-b376-88fe963b07db
To: /workspace/WM-JEPA/PLDM_hieral/pldm_envs/wall/presaved_datasets/wall_data.tar.gz
100%|████████████████████████████████████████| 535M/535M [00:16<00:00, 32.7MB/s]
ds_size_1269_no_images.npz
ds_size_1500K_no_images.npz
ds_size_20312_no_images.npz
ds_size_325K_no_images.npz
ds_size_5078_no_images.npz
ds_size_634_no_images.npz
ds_size_81250_no_images.npz
good_quality_data_no_images.npz
len_17_no_images.npz
len_33_no_images.npz
len_65_no_images.npz
noise_mix_0.001_no_images.npz
noise_mix_0.01_no_images.npz
noise_mix_0.02_no_images.npz
noise_mix_0.04_no_images.npz
noise_mix_0.08_no_images.npz
noise_mix_0.16_no_images.npz
random_trajectories_no_images.npz
wc_rate_0.npz
/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml

In [6]:
!python PLDM_hieral/generate_wall_trials.py \
  --config PLDM_hieral/configs/tworooms_l1.yaml \
  --output_train PLDM_hieral/wall_trials_train.npz \
  --output_eval PLDM_hieral/wall_trials_eval.npz \
  --n_eval 100 \
  --seed 42


/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
Saved train trials to PLDM_hieral/wall_trials_train.npz
Saved eval trials to PLDM_hieral/wall_trials_eval.npz


## Run L1 2M baseline (no L2)
Original-size L1-only model (no parameter matching).


In [20]:
!python PLDM_hieral/generate_wall_trials.py \
  --config PLDM_hieral/configs/tworooms_l1.yaml \
  --output_train PLDM_hieral/wall_trials_train.npz \
  --output_eval PLDM_hieral/wall_trials_eval.npz \
  --n_eval 100 \
  --seed 42

!CUDA_VISIBLE_DEVICES=0 python PLDM_hieral/run_tworooms_compare.py \
  --config_l1 PLDM_hieral/configs/tworooms_l1.yaml \
  --mode l1 \
  --output_root {OUTPUT_ROOT} \
  --epochs 3

/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
Saved train trials to PLDM_hieral/wall_trials_train.npz
Saved eval trials to PLDM_hieral/wall_trials_eval.npz
/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that insta

## Run L1 (2M) vs L2 comparison
Runs L1-only (`tworooms_l1.yaml`) then L2 (`tworooms_l2.yaml`) from scratch.


In [ ]:
!python PLDM_hieral/generate_wall_trials.py \
  --config PLDM_hieral/configs/tworooms_l1.yaml \
  --output_train PLDM_hieral/wall_trials_train.npz \
  --output_eval PLDM_hieral/wall_trials_eval.npz \
  --n_eval 100 \
  --seed 42

!CUDA_VISIBLE_DEVICES=0 python PLDM_hieral/run_tworooms_compare.py \
  --config_l1 PLDM_hieral/configs/tworooms_l1.yaml \
  --config_l2 PLDM_hieral/configs/tworooms_l2.yaml \
  --output_root {OUTPUT_ROOT} \
  --l2_from_scratch \
  --mode both \
  --epochs 3


/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
Saved train trials to PLDM_hieral/wall_trials_train.npz
Saved eval trials to PLDM_hieral/wall_trials_eval.npz
/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that insta

Outputs are saved to Google Drive under `/content/drive/MyDrive/PLDM_2mvs6m`.


In [ ]:
# Outputs are already in Drive.
!ls -la {OUTPUT_ROOT}
